In [ ]:
pip install pandas numpy matplotlib seaborn scikit-learn statsmodels plotly plotly_express

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
import zipfile
import os
from sklearn.model_selection import train_test_split


In [4]:
import zipfile
import os

# Path to the main ZIP file
main_zip_path = 'WaterDistributionDataset.zip'

# Extract main ZIP file
with zipfile.ZipFile(main_zip_path, 'r') as zip_ref:
    zip_ref.extractall('WaterDistributionDataset')  # Extract to a directory

# Path to the sub-ZIP files inside the extracted directory
sub_zip_paths = [
    'WaterDistributionDataset/WaDi.A1_9 Oct 2017.zip',
    'WaterDistributionDataset/WaDi.A2_19 Nov 2019.zip'
]

# Unzip each of the nested ZIP files
for sub_zip_path in sub_zip_paths:
    if os.path.exists(sub_zip_path):
        with zipfile.ZipFile(sub_zip_path, 'r') as zip_ref:
            zip_ref.extractall(os.path.splitext(sub_zip_path)[0])  # Extract to a folder named after the ZIP file
        print(f"Extracted: {sub_zip_path}")
    else:
        print(f"File not found: {sub_zip_path}")


Extracted: WaterDistributionDataset/WaDi.A1_9 Oct 2017.zip
Extracted: WaterDistributionDataset/WaDi.A2_19 Nov 2019.zip


In [10]:
# Load the datasets
wadi_14days_path = 'WaterDistributionDataset/WaDi.A2_19 Nov 2019/WADI.A2_19 Nov 2019/WADI_14days_new.csv'
wadi_attack_labels_path = 'WaterDistributionDataset/WaDi.A2_19 Nov 2019/WADI.A2_19 Nov 2019/WADI_attackdataLABLE.csv'

# Read the CSV files into pandas DataFrames
df_wadi_14days = pd.read_csv(wadi_14days_path)
df_attack_labels = pd.read_csv(wadi_attack_labels_path, header=1)

# Display the first few rows of each dataframe to understand the structure of the data
print("WADI_14days_new.csv preview:")
df_wadi_14days.head(20)



WADI_14days_new.csv preview:


,Row,Date,Time,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,...,3_MV_001_STATUS,3_MV_002_STATUS,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW
0,1,9/25/2017,00:00.0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,...,1,1,1,1,1,1,1,67.9651,1,0.68
1,2,9/25/2017,00:01.0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,...,1,1,1,1,1,1,1,67.9651,1,0.68
2,3,9/25/2017,00:02.0,171.155,0.619473,11.5759,504.645,0.318319,0.001157,0,...,1,1,1,1,1,1,1,67.9651,1,0.68
3,4,9/25/2017,00:03.0,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,...,1,1,1,1,1,1,1,67.1948,1,0.68
4,5,9/25/2017,00:04.0,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,...,1,1,1,1,1,1,1,67.1948,1,0.68
5,6,9/25/2017,00:05.0,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,...,1,1,1,1,1,1,1,67.1948,1,0.68
6,7,9/25/2017,00:06.0,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,...,1,1,1,1,1,1,1,67.1948,1,0.68
7,8,9/25/2017,00:07.0,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,...,1,1,1,1,1,1,1,67.1948,1,0.68
8,9,9/25/2017,00:08.0,171.155,0.607477,11.5725,504.673,0.318438,0.001207,0,...,1,1,1,1,1,1,1,67.1948,1,0.68
9,10,9/25/2017,00:09.0,171.151,0.613478,11.5735,504.701,0.318495,0.001260,0,...,1,1,1,1,1,1,1,63.9867,1,0.68


In [11]:
print("\nWADI_attackdataLABLE.csv preview:")
df_attack_labels.head(20)



WADI_attackdataLABLE.csv preview:


,Row,Date,Time,1_AIT_001_PV,1_AIT_002_PV,1_AIT_003_PV,1_AIT_004_PV,1_AIT_005_PV,1_FIT_001_PV,1_LS_001_AL,...,3_MV_002_STATUS,3_MV_003_STATUS,3_P_001_STATUS,3_P_002_STATUS,3_P_003_STATUS,3_P_004_STATUS,LEAK_DIFF_PRESSURE,PLANT_START_STOP_LOG,TOTAL_CONS_REQUIRED_FLOW,"Attack LABLE (1:No Attack, -1:Attack)"
0,1.0,10/9/17,00:00.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,1
1,2.0,10/9/17,00:01.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,1
2,3.0,10/9/17,00:02.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,1
3,4.0,10/9/17,00:03.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,1
4,5.0,10/9/17,00:04.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,1
5,6.0,10/9/17,00:05.0,164.210,0.529486,11.9972,482.480,0.331167,0.001273,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,62.6226,1.0,0.39,1
6,7.0,10/9/17,00:06.0,164.212,0.547483,11.9946,482.474,0.331282,0.001070,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,62.6153,1.0,0.39,1
7,8.0,10/9/17,00:07.0,164.212,0.547483,11.9946,482.474,0.331282,0.001070,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,62.6153,1.0,0.39,1
8,9.0,10/9/17,00:08.0,164.212,0.547483,11.9946,482.474,0.331282,0.001070,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,62.6153,1.0,0.39,1
9,10.0,10/9/17,00:09.0,164.212,0.547483,11.9946,482.474,0.331282,0.001070,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,62.6153,1.0,0.39,1


In [12]:
#shape of the dataframes
print("\nShape of WADI_14days_new.csv:", df_wadi_14days.shape)
print("Shape of WADI_attackdataLABLE.csv:", df_attack_labels.shape)


Shape of WADI_14days_new.csv: (784571, 130)
Shape of WADI_attackdataLABLE.csv: (172803, 131)


In [16]:
# ceckin info of the dataframes
print("Info of WADI_14days_new.csv:", df_wadi_14days.info())
print("\nInfo of WADI_attackdataLABLE.csv:", df_attack_labels.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784571 entries, 0 to 784570
Columns: 130 entries, Row to TOTAL_CONS_REQUIRED_FLOW
dtypes: float64(71), int64(57), object(2)
memory usage: 778.2+ MB
Info of WADI_14days_new.csv: None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172803 entries, 0 to 172802
Columns: 131 entries, Row  to Attack LABLE (1:No Attack, -1:Attack)
dtypes: float64(128), int64(1), object(2)
memory usage: 172.7+ MB

Info of WADI_attackdataLABLE.csv: None


In [18]:
# Check the columns of the dataframes to understand their structure
print("\nColumns in WADI_14days_new.csv:", df_wadi_14days.columns.tolist())
print("\nColumns in WADI_attackdataLABLE.csv:", df_attack_labels.columns.tolist())


Columns in WADI_14days_new.csv: ['Row', 'Date', 'Time', '1_AIT_001_PV', '1_AIT_002_PV', '1_AIT_003_PV', '1_AIT_004_PV', '1_AIT_005_PV', '1_FIT_001_PV', '1_LS_001_AL', '1_LS_002_AL', '1_LT_001_PV', '1_MV_001_STATUS', '1_MV_002_STATUS', '1_MV_003_STATUS', '1_MV_004_STATUS', '1_P_001_STATUS', '1_P_002_STATUS', '1_P_003_STATUS', '1_P_004_STATUS', '1_P_005_STATUS', '1_P_006_STATUS', '2_DPIT_001_PV', '2_FIC_101_CO', '2_FIC_101_PV', '2_FIC_101_SP', '2_FIC_201_CO', '2_FIC_201_PV', '2_FIC_201_SP', '2_FIC_301_CO', '2_FIC_301_PV', '2_FIC_301_SP', '2_FIC_401_CO', '2_FIC_401_PV', '2_FIC_401_SP', '2_FIC_501_CO', '2_FIC_501_PV', '2_FIC_501_SP', '2_FIC_601_CO', '2_FIC_601_PV', '2_FIC_601_SP', '2_FIT_001_PV', '2_FIT_002_PV', '2_FIT_003_PV', '2_FQ_101_PV', '2_FQ_201_PV', '2_FQ_301_PV', '2_FQ_401_PV', '2_FQ_501_PV', '2_FQ_601_PV', '2_LS_001_AL', '2_LS_002_AL', '2_LS_101_AH', '2_LS_101_AL', '2_LS_201_AH', '2_LS_201_AL', '2_LS_301_AH', '2_LS_301_AL', '2_LS_401_AH', '2_LS_401_AL', '2_LS_501_AH', '2_LS_501_

In [ ]:
# check for missing values in the dataframes
print("\nMissing values in WADI_14days_new.csv:", df_wadi_14days.isnull().sum())
print("\nMissing values in WADI_attackdataLABLE.csv:", df_attack_labels.isnull().sum())
